<a href="https://colab.research.google.com/github/juanpeter-yuune/email_spam_using_BERT/blob/master/JUAN_BERT_email_classification_handle_imbalance_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-text
!pip install tensorflow==2.15.0 tensorflow-hub==0.16.1 tensorflow-text==2.15.0

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from matplotlib import pyplot as plt
import seaborn as sn
from sklearn.metrics import classification_report, confusion_matrix, matthews_corrcoef, roc_auc_score
from tensorflow.keras import layers, Model

In [3]:
df = pd.read_csv("emailspam.csv")
df.head(5)

,Category,Message
0,"ham,""Go until jurong point, crazy.. Available ...",NaN
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,"ham,""Nah I don't think he goes to usf, he live...",NaN


In [4]:
df.groupby('Category').describe()

Message         \
                                                     count unique   
Category                                                            
 here's a special discount code for your next p...       1      1   
 please contact us immediately.                          1      1   
 thank you for subscribing to our newsletter. H...       1      1   
 we'll achieve great things!                             1      1   
 we'll make a difference!                                1      1   
...                                                    ...    ...   
spam,"ou are guaranteed the latest Nokia Phone,...       0      0   
spam,"our mobile number has won Â£5000, to clai...       0      0   
spam,"pdate_Now - Double mins and 1000 txts on ...       0      0   
spam,"tddnewsletter@emc1.co.uk (More games from...       0      0   
spam,"u r subscribed 2 TEXTCOMP 250 wkly comp. ...       0      0   

                                                                                                       \
                                                                                                  top   
Category                                                                                                
 here's a special discount code for your next p...                               Dear valued customer   
 please contact us immediately.                     Your account login was successful. If this was...   
 thank you for subscribing to our newsletter. H...                                        Dear [Name]   
 we'll achieve great things!                        We're pleased to announce our partnership with...   
 we'll make a difference!                           We're thrilled to announce our partnership wit...   
...                                                                                               ...   
spam,"ou are guaranteed the latest Nokia Phone,...                                                NaN   
spam,"our mobile number has won Â£5000, to clai...                                                NaN   
spam,"pdate_Now - Double mins and 1000 txts on ...                                                NaN   
spam,"tddnewsletter@emc1.co.uk (More games from...                                                NaN   
spam,"u r subscribed 2 TEXTCOMP 250 wkly comp. ...                                                NaN   

                                                         
                                                   freq  
Category                                                 
 here's a special discount code for your next p...    1  
 please contact us immediately.                       1  
 thank you for subscribing to our newsletter. H...    1  
 we'll achieve great things!                          1  
 we'll make a difference!                             1  
...                                                 ...  
spam,"ou are guaranteed the latest Nokia Phone,...  NaN  
spam,"our mobile number has won Â£5000, to clai...  NaN  
spam,"pdate_Now - Double mins and 1000 txts on ...  NaN  
spam,"tddnewsletter@emc1.co.uk (More games from...  NaN  
spam,"u r subscribed 2 TEXTCOMP 250 wkly comp. ...  NaN  

[1237 rows x 4 columns]

In [5]:
df['Category'].value_counts()

,count
Category,
ham,3778
spam,577
"ham,""Sorry, I'll call later""",29
"ham,""Wen ur lovable bcums angry wid u, dnt take it seriously.. Coz being angry is d most childish n true way of showing deep affection, care n luv!.. kettoda manda... Have nice day da.""",4
"ham,""Say this slowly.? GOD,I LOVE YOU &amp",4
...,...
"ham,""Dude. What's up. How Teresa. Hope you have been okay. When i didnt hear from these people, i called them and they had received the package since dec &lt",1
"ham,""Now, whats your house # again ? And do you have any beer there ?""",1
"ham,""Aight, I'll ask a few of my roommates""",1


In [6]:
df_spam = df[df['Category']=='spam']
df_spam.shape

(577, 2)

In [7]:
df_ham = df[df['Category']=='ham']
df_ham.shape

(3778, 2)

In [8]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(577, 2)

In [9]:
df_balanced = pd.concat([df_ham_downsampled, df_spam])
df_balanced.shape

(1154, 2)

In [10]:
df_balanced['Category'].value_counts()

,count
Category,
ham,577
spam,577


In [11]:
df_balanced['spam']=df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)
df_balanced.sample(5)

,Category,Message,spam
2573,ham,Yup i'm elaborating on the safety aspects and ...,0
4805,spam,Check Out Choose Your Babe Videos @ sms.shsex....,1
4555,spam,Wanna have a laugh? Try CHIT-CHAT on your mobi...,1
2260,spam,88066 FROM 88066 LOST 3POUND HELP,1
1699,ham,Please ask mummy to call father,0


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'],df_balanced['spam'], stratify=df_balanced['spam'])

In [13]:
# Count the number of samples in each class for the training set
train_counts = y_train.value_counts()
print("Training data class distribution:")
print(train_counts)

# Count the number of samples in each class for the testing set
test_counts = y_test.value_counts()
print("\nTesting data class distribution:")
print(test_counts)


Training data class distribution:
spam
0    433
1    432
Name: count, dtype: int64

Testing data class distribution:
spam
1    145
0    144
Name: count, dtype: int64


In [14]:
X_train.head(4)

,Message
1090,Please da call me any mistake from my side sor...
5663,Congratulations! You've been selected for a fr...
5558,Upgrade to our premium plan for exclusive acce...
4051,Moby Pub Quiz.Win a Â£100 High Street prize if...


In [15]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [16]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up",
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8435168 , -0.5132724 , -0.8884571 , ..., -0.7474884 ,
        -0.7531473 ,  0.91964483],
       [-0.87208354, -0.50543964, -0.94446665, ..., -0.85847497,
        -0.7174535 ,  0.88082975]], dtype=float32)>

In [17]:
e = get_sentence_embeding([
    "free",
    "winner",
    "click here",
    "exclusive deal",
    "limited time offer",
    "act now",
    "urgent",
    "money",
    "cash",
    "investment",
    "guaranteed",
    "make money",
    "credit card",
    "debt relief",
    "account",
    "login",
    "verify",
    "update",
    "risk-free",
    "trial",
    "satisfaction guaranteed",
    "meeting",
    "project",
    "update",
    "collaboration",
    "report",
    "schedule",
    "hello",
    "thank you",
    "regards",
    "please",
    "feedback",
    "follow-up",
    "contract",
    "invoice",
    "purchase order",
    "quote",
    "resume",
    "email",
    "message",
    "reply",
    "information",
    "data",
    "send",
    "receive",
    "ask",
    "call",
    "security alert",
    "your payment",
    "unsubscribe",
    "spam"
]
)

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[1]])

array([[0.96217215]], dtype=float32)

In [19]:
cosine_similarity([e[0]],[e[3]])

array([[0.81732816]], dtype=float32)

In [20]:
cosine_similarity([e[3]],[e[4]])

array([[0.9930567]], dtype=float32)

In [23]:
from sklearn.neighbors import KNeighborsClassifier
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)


# Neural network layers
model_clf = KNeighborsClassifier(n_neighbors=5, leaf_size=30, metric_params=None)


# Use inputs and outputs to construct a final model
# Instead of using the undefined variable 'l', we will use the 'pooled_output' from the BERT encoder.
# You can choose to use 'sequence_output' instead, depending on your needs.
model = tf.keras.Model(inputs=[text_input], outputs = [outputs['pooled_output']])

In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_mask': (None, 128)   0         ['text[0][0]']                
                             , 'input_type_ids': (None,                                           
                              128),                                                               
                              'input_word_ids': (None,                                            
                             128)}                                                                
                                                                                              

In [25]:
len(X_train)

865

In [26]:
# Custom specificity metric
def specificity(y_true, y_pred):
    true_negatives = tf.math.reduce_sum(tf.cast((y_true == 0) & (y_pred < 0.5), tf.float32))
    false_positives = tf.math.reduce_sum(tf.cast((y_true == 0) & (y_pred >= 0.5), tf.float32))
    specificity_value = true_negatives / (true_negatives + false_positives + tf.keras.backend.epsilon())
    return specificity_value

# Custom MCC metric
def matthews_correlation_coefficient(y_true, y_pred):
    y_pred_pos = tf.round(y_pred)
    y_pred_neg = 1 - y_pred_pos

    y_true_pos = tf.round(y_true)
    y_true_neg = 1 - y_true_pos

    tp = tf.reduce_sum(y_true_pos * y_pred_pos)
    tn = tf.reduce_sum(y_true_neg * y_pred_neg)

    fp = tf.reduce_sum(y_true_neg * y_pred_pos)
    fn = tf.reduce_sum(y_true_pos * y_pred_neg)

    numerator = (tp * tn) - (fp * fn)
    denominator = tf.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    mcc = numerator / (denominator + tf.keras.backend.epsilon())
    return mcc



In [29]:
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf

# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
# Instead of defining a KNeighborsClassifier here,
# we'll add a dense layer for classification on top of the BERT embeddings.
# This layer will output a single value for binary classification.
clf_output = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(outputs['pooled_output'])

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs=[clf_output])

# Custom specificity metric
def specificity(y_true, y_pred):
    true_negatives = tf.math.reduce_sum(tf.cast((y_true == 0) & (y_pred < 0.5), tf.float32))
    false_positives = tf.math.reduce_sum(tf.cast((y_true == 0) & (y_pred >= 0.5), tf.float32))
    specificity_value = true_negatives / (true_negatives + false_positives + tf.keras.backend.epsilon())
    return specificity_value

# Custom MCC metric
def matthews_correlation_coefficient(y_true, y_pred):
    y_pred_pos = tf.round(y_pred)
    y_pred_neg = 1 - y_pred_pos

    y_true_pos = tf.round(y_true)
    y_true_neg = 1 - y_true_pos

    tp = tf.reduce_sum(y_true_pos * y_pred_pos)
    tn = tf.reduce_sum(y_true_neg * y_pred_neg)

    fp = tf.reduce_sum(y_true_neg * y_pred_pos)
    fn = tf.reduce_sum(y_true_pos * y_pred_neg)

    numerator = (tp * tn) - (fp * fn)
    denominator = tf.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    mcc = numerator / (denominator + tf.keras.backend.epsilon())
    return mcc

# Compile model with custom metrics
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),  # Sensitivity
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.AUC(name='prc', curve='PR'),  # Precision-Recall AUC (optional)
    specificity,  # Custom specificity metric
    matthews_correlation_coefficient  # Custom MCC metric
]

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)

# Train the model and store the history
history = model.fit(X_train, y_train, epochs=10)

# Print the performance metrics from the history
for metric in history.history:
    print(f'{metric}: {history.history[metric][-1]}')

Epoch 1/10
28/28 [==============================] - 452s 16s/step - loss: 0.6009 - accuracy: 0.7457 - precision: 0.7314 - recall: 0.7755 - auc: 0.8057 - prc: 0.7528 - specificity: 0.7255 - matthews_correlation_coefficient: 0.4986
Epoch 2/10
28/28 [==============================] - 438s 16s/step - loss: 0.5085 - accuracy: 0.8058 - precision: 0.8084 - recall: 0.8009 - auc: 0.8820 - prc: 0.8696 - specificity: 0.7862 - matthews_correlation_coefficient: 0.6045
Epoch 3/10
28/28 [==============================] - 450s 16s/step - loss: 0.4564 - accuracy: 0.8301 - precision: 0.8051 - recall: 0.8704 - auc: 0.9064 - prc: 0.8984 - specificity: 0.8011 - matthews_correlation_coefficient: 0.6509
Epoch 4/10
28/28 [==============================] - 448s 16s/step - loss: 0.4190 - accuracy: 0.8393 - precision: 0.8307 - recall: 0.8519 - auc: 0.9214 - prc: 0.9234 - specificity: 0.8351 - matthews_correlation_coefficient: 0.6702
Epoch 5/10
28/28 [==============================] - 451s 16s/step - loss: 0.3889